# Wikimedia data
Wikimedia Downloads: Analytics Datasets Info about Pageviews, mediacounts and unique devices:
Pageviews since may 2015:
https://dumps.wikimedia.org/other/pageviews/[YEAR]/[YEAR]-[2-DIGIT-MONTH]/projectviews-YYYYMMDD-HH0000

In [45]:
from datetime import datetime, timedelta

def time_range(init, end):
    while init<end:
        yield init
        init+=timedelta(hours=1)   
    
def explode_time(date):
    return {
        "year": date.year,
        "month": date.month,
        "day": date.day,
        "hour": date.hour,
    }

def gen_url(exploded_date):
    return "https://dumps.wikimedia.org/other/pageviews/{year}/{year}-{month:02d}/projectviews-{year}{month:02d}{day:02d}-{hour:02d}0000".format(**exploded_date)

exploded_dates = map(explode_time, time_range(datetime(2016,1,1), datetime(2017,1,31)))
urls = map(gen_url, exploded_dates)

#list(urls)